<a href="https://colab.research.google.com/github/Rocco000/OncoVision/blob/main/Scripts/DatasetLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import the libraries**

In [ ]:
"""#Import the libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

print(torch.__version__)"""

**Import data from Google Drive**

In [ ]:
"""from google.colab import drive
drive.mount('/content/drive') #connect to Google Drive"""

**Try to read the dataset images**

In [ ]:
"""from google.colab.patches import cv2_imshow #because Google Colab doesn't support cv2.imshow(), it causes Jupyter session to crash
import cv2
import random

file_path = '/content/drive/MyDrive/SE4AI/Data/Datasets/Dataset1/benign/melanoma_0.jpg'
image = cv2.imread(file_path)
cv2_imshow(image)
print("Width: ",image.shape[1])
print(image)

def resize_image(input_image_path):
  resized_image = cv2.resize(input_image_path, (600, 450))
  cv2_imshow(resized_image)
  return resized_image

resized = resize_image(image)
#Transform the image into a tensor
transform = transforms.ToTensor()
tensor_image = transform(resized)
print(tensor_image)"""

**Load dataset**

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

def get_dataset(data_path, percent_train, percent_valid, batch_size):
  # Define the transformation to apply to the images
  transform = transforms.Compose([
      transforms.Resize((600, 450)),  # Resize the images to 600x450
      transforms.ToTensor()  # Convert the images to tensors
  ])

  # Create the ImageFolder dataset and apply the transformation
  dataset = ImageFolder(root=data_path, transform=transform)
  print("Labels:")
  print(dataset.class_to_idx)

  #Compute the number of sample for each set
  train_size = int(percent_train * len(dataset))
  valid_size = int(percent_valid * len(dataset))
  test_size = len(dataset) - train_size - valid_size

  #Split the dataset into train, validation, and test sets in a random manner
  train_set, valid_set, test_set = random_split(dataset, [train_size, valid_size, test_size])

  #Define a DatLoder for each set
  #num_workers parallelize data loading. Each subprocess loads a batch of data independently and in parallel
  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True) #in this case it shuffle the date for each epoch
  valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)
  test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

  return train_loader, valid_loader, test_loader

def plot_samples(data_loader, to_print, folder, flag):
  num_class_instances = dict({"0":0,"1":0})
  for imgs, labels in data_loader:
    for l in labels:
      #l.item() to convert the tensor in a scalar value
      str_label = str(l.item())
      num_class_instances[str_label] += 1

  class_labels = list(num_class_instances.keys())
  class_counts = list(num_class_instances.values())

  # Create a bar plot
  plt.bar(class_labels, class_counts)

  title = 'Number of samples per Class ('+to_print+")"
  plt.xlabel('Class Label')
  plt.ylabel('Num instances')
  plt.title(title)
  if flag:
    plt.savefig('/content/drive/MyDrive/SE4AI/Model/'+folder+'/plot'+to_print+'.png')
  else:
    plt.savefig('/content/drive/MyDrive/LinkToOncoVision/SE4AI/Model/'+folder+'/plot'+to_print+'.png')
  plt.show()
  print("Number of samples per class (",to_print,"): ",num_class_instances)

#train_loader, valid_loader, test_loader = get_dataset("/content/drive/MyDrive/SE4AI/Data/Datasets/Dataset2/", 0.7,0.2,64)
#
#i = 0
#for images, labels in train_loader:
#  count = 0
#  i = i+1
#  for image in images:
#    count = count +1
#  print("Batch [",i,"] has ",count," samples")
#
#nuovo_train = DataLoader(train_loader.dataset, batch_size=32, shuffle=True, num_workers=2, drop_last=True)
#i = 0
#for images, labels in nuovo_train:
#  count = 0
#  i = i+1
#  for image in images:
#    count = count +1
#  print("Batch [",i,"] has ",count," samples")
#
#plot_samples(train_loader, "TRAIN SET")